# Librerias

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

2025-09-14 04:01:06.419026: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-14 04:01:06.526512: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/g

In [2]:
!pip3 install --upgrade wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 4.3 MB/s  0:00:04 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [wandb]m 9/10 [wandb]ic]k]


### Sublibrerias de tensorflow -keras-

In [3]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from keras import layers
from keras import initializers
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers

### Abrimos wandb

In [6]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
wandb.login()

True

### Hiper-parametros

In [7]:
lr = 0.005 #Learning rate
epochs = 30
bs = 120 #batch size
loss = "categorical_crossentropy"

### inicialización de wandb

In [8]:
run = wandb.init(
    # set the wandb project where this run will be logged
    project="Deeplearning_Digit_Recognition",

    # track hyperparameters and run metadata with wandb.config
    config={
        "layer_1": 512,
        "activation_1": "sigmoid",
        "activation_2": "softmax",
        "w_initializer": "RandomNormal",
        "optimizer": "RMSprop",
        "loss": loss,
        "metric": "accuracy",
        "epoch": epochs,
        "batch_size": bs
    }
)

### Carga de datos

In [9]:
dataset=mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


### Datos de entrenamiento y prueba

In [10]:
(x_train, y_train), (x_test, y_test) = dataset

### Verificamos la forma de los datos

In [11]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(60000, 28, 28) (10000, 28, 28) (60000,) (10000,)


### Se aplanan las imagenes y se normalizan

In [12]:
entrada = int(x_train.shape[1]*x_train.shape[2])
stddev = 1/np.sqrt(entrada)

x_train = x_train.reshape(int(x_train.shape[0]), entrada)
x_test = x_test.reshape(int(x_test.shape[0]), entrada)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.
x_test /= 255.

### Se aplica one-hot encoding sobre las categorias

In [13]:
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Creación de la red neuronal

In [14]:
model = keras.Sequential(
    [
        layers.Dense(512, activation = "sigmoid", name = "layer1", 
                     kernel_initializer=initializers.RandomNormal(stddev = stddev)),
        layers.Dense(num_classes, activation = "softmax", name = "output"),
    ]
) #Arquitectura de la red neuronal
model.compile(loss = loss, optimizer = RMSprop(learning_rate = lr),metrics = ["accuracy"]) #Función de costo
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
history = model.fit(x_train, y_train,
                    batch_size = bs,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test),
                    callbacks=[
                        WandbMetricsLogger(log_freq=5),
                        WandbModelCheckpoint("models/model.keras")
                    ])
run.finish()

Epoch 1/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9034 - loss: 0.3273 - val_accuracy: 0.9562 - val_loss: 0.1373
Epoch 2/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9646 - loss: 0.1161 - val_accuracy: 0.9587 - val_loss: 0.1285
Epoch 3/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9785 - loss: 0.0716 - val_accuracy: 0.9764 - val_loss: 0.0736
Epoch 4/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9844 - loss: 0.0503 - val_accuracy: 0.9784 - val_loss: 0.0685
Epoch 5/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9880 - loss: 0.0367 - val_accuracy: 0.9797 - val_loss: 0.0659
Epoch 6/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9922 - loss: 0.0252 - val_accuracy: 0.9819 - val_loss: 0.0668
Epoch 7/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9945 - loss: 0.0176 - val_accuracy: 0.9817 - val_loss: 0.0622
Epoch 8/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9958 - loss: 0.0137 - val_accuracy: 0.

batch/accuracy,▁▂▄▇▇▇█▇████████████████████████████████
batch/batch_step,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇██
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▅▆▇▇▇████████████████████████
epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▂▆▇▇▇▇█▇█████████████████████
epoch/val_loss,█▇▂▂▁▁▁▁▁▁▁▁▂▂▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂
batch/accuracy,1


In [16]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
a = model.predict(x_test)
print(a.shape)
print(a[1])
print("resultado correcto:")
print(y_test[1])

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9841 - loss: 0.0712
Test loss: 0.0712357833981514
Test accuracy: 0.9840999841690063
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
(10000, 10)
[7.2453377e-15 1.3830694e-10 1.0000000e+00 4.6925064e-12 1.0656875e-18
 8.2684612e-14 4.8602881e-13 7.9014987e-20 9.7751651e-10 2.1836956e-20]
resultado correcto:
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [17]:
#Para guardar el modelo en disco
model.save("red1.keras")